In [1]:
# %cd ..
!nvidia-smi

# Mount a google drive with the person_detection repository inside from here https://github.com/Yannick947/person_detection
from google.colab import drive
drive.mount('/content/drive')

Fri Aug 21 10:56:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install gdown
!pip install tensorflow-gpu==2.2 --user
!pip install keras==2.3.1
!pip install keras_resnet
!pip install efficientnet

     |████████████████████████████████| 516.2MB 27kB/s 
     |████████████████████████████████| 3.0MB 70.0MB/s 
     |████████████████████████████████| 460kB 58.8MB/s 
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.2.0 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     |████████████████████████████████| 378kB 90kB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20487 sha256=4216fd8419d13

In [3]:
%cd /content/sample_data/
!git clone https://github.com/Yannick947/WiderPerson.git -b master --single-branch
%cd ../../

/content/sample_data
Cloning into 'WiderPerson'...
remote: Enumerating objects: 13396, done.
remote: Total 13396 (delta 0), reused 0 (delta 0), pack-reused 13396
Receiving objects: 100% (13396/13396), 964.79 MiB | 42.13 MiB/s, done.
Checking out files: 100% (13386/13386), done.
/


In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
from time import gmtime, strftime

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


# Training



In [5]:
PRETRAINED_RES50_VANILLA = '/content/drive/My\ Drive/person_detection/keras-retinanet/snapshots/resnet50_vanilla.h5'
PRETRAINED_RES152_VANILLA = './snapshots/resnet152_vanilla_oid.h5'
SNAPSHOT_PATH = '../drive/My\ Drive/person_detection/keras-retinanet/snapshots/'
#resnet50, map71, standard resize, no augmentation, lr 1e-5
PRETRAINED_MODEL_WI = '../drive/My\ Drive/person_detection/keras-retinanet/snapshots/17-04-2020_07465_resnet50_08_1.h5'
logdir = '../drive/My\ Drive/person_detection/keras-retinanet/tensorboard/{}_1'.format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

In [ ]:
%cd /content/drive/My\ Drive/person_detection/keras-retinanet/
for _ in range(5):

    logdir = '/content/drive/My\ Drive/person_detection/keras-retinanet/tensorboard_WiderPerson/{}_1'.format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
    aug = np.random.uniform(low=0.0, high=0.5)
    vis_aug = np.random.uniform(low=0.0, high=0.5)
    lr = np.random.uniform(low=1e-4, high=1e-6)

    print('Training for following parameters: \nVisual augmentation factor: {} \nAugmentation factor: {}'.format(vis_aug, aug))
    !python keras_retinanet/bin/train.py  --lr $lr\
                                        --batch-size 2\
                                        --epochs 30 \
                                        --backbone 'resnet50'\
                                        --weighted-average\
                                        --tensorboard-dir {logdir}\
                                        --compute-val-loss\
                                        --workers 0\
                                        --max-queue-size 1\
                                        --no-snapshots\
                                        --visual-aug-factor $vis_aug\
                                        --augmentation-factor $aug\
                                        --config ./config.ini\
                                        csv ./annotations/annotations_train.csv ./annotations/classes.csv \
                                        --val-annotations ./annotations/annotations_test.csv \

%cd ../../../../../